# Imptox various stuff

- Copy and manage images

## Copy images into library

In [ ]:
import glob
import shutil
import os
from PIL import Image

import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib import pyplot as plt

In [ ]:



src_dir = "C:/Users/TWG/switchdrive/H2020_IMPTOX_CI4CB/7_Applications_SampleData/1_imageSample/wetransfer_for-ai-team_2022-02-20_1821"
dst_dir = "C:/Users/TWG/switchdrive/H2020_IMPTOX_CI4CB/7_Applications_SampleData/1_imageSample/1_raw_library/"



#for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
for root, subdirs, files in os.walk(src_dir):
    
    print(f"\n\n------ Sub-dirs: {subdirs}----------")
    print(f"\n====== Root: {root}")
    print(f"\nFile: {files}")
    
    # name of the parent directory
    exp_name = os.path.basename(os.path.normpath(root)).replace(" ", "_")
    
    for file in files:
        
        # Complete file path
        filepath = root + "/" + file
        
        # Separate base from extension
        base, extension = os.path.splitext(file)
        
        print(f"Treating file {base}{extension}")
        
        # Trim filename spaces
        new_base = base.replace(" ", "_")
        destination = ""
        
        if(extension == ".jpg"):
            # Copy into jpg dst_dir
            print("jpg")
            destination = dst_dir + "jpg/" + exp_name + "-" + new_base + extension
            print(f"Destination: {destination}")
            shutil.copy(filepath, destination)
        elif(extension == ".tif"):
            # Copy into tif dst_dir
            print("tif")
            destination = dst_dir + "tif/" + exp_name + "-" + new_base + extension
            print(f"Destination: {destination}")
            shutil.copy(filepath, destination)
        
    
    
    
    
    
    
    
    
    
    
    

# Images quality and resolution

Exploration of the available images



In [ ]:
img_dir  = "C:/Users/TWG/switchdrive/H2020_IMPTOX_CI4CB/7_Applications_SampleData/1_imageSample/1_raw_library/tif"

tif_shapes = []
tif_filelist = []


for root, subdirs, files in os.walk(img_dir):
    for file in files:
        
        # Complete file path
        filepath = root + "/" + file
        
        # Separate base from extension
        base, extension = os.path.splitext(file)
        
        #print(f"Treating file {base}{extension}")
        
        # Open file with Pillow
        with Image.open(filepath) as im:
            #print(im.size)
            #im.show()
            imarray = np.array(im)
            #print(imarray.shape)
            tif_shapes.append(imarray.shape)
            tif_filelist.append(filepath)
            

# Create dataframe with image resolutions from numpy array (3D)
df = pd.DataFrame(tif_shapes)


        

In [ ]:
# Count different image resolution
df.groupby(df.columns.tolist(),as_index=False).size()

In [ ]:
plt.rcParams['figure.figsize'] = [20, 15]
for idx, im_path in enumerate(tif_filelist):
    if idx != 0:
        break
    with Image.open(im_path) as im:
        # Takes a bit of time to display.
        #display(im)
        img = cv.imread(im_path,0)
        edges = cv.Canny(img,60,60)
        
        plt.imshow(img,cmap = 'gray')
        plt.title('Original Image'), plt.xticks([]), plt.yticks([])
        plt.show()
        
        plt.imshow(edges,cmap = 'gray')
        plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
        plt.show()
        

### Test, actually good to extract filter

In [ ]:
for idx, im_path in enumerate(tif_filelist):
   # if idx != 0:
   #     break
    with Image.open(im_path) as im:
        # Takes a bit of time to display.
        #display(im)
        gray = img = cv.imread(im_path,0)
        
        edges = cv.Canny(img,60,60)
        
        print(img.shape)

        #gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
        
        plt.imshow(img,cmap = 'gray')
        plt.title('Grey thresholded Image'), plt.xticks([]), plt.yticks([])
        plt.show()
        
        # noise removal
        kernel = np.ones((3,3),np.uint8)
        opening = cv.morphologyEx(thresh,cv.MORPH_OPEN,kernel, iterations = 2)
        # sure background area
        sure_bg = cv.dilate(opening,kernel,iterations=3)
        # Finding sure foreground area
        dist_transform = cv.distanceTransform(opening,cv.DIST_L2,5)
        ret, sure_fg = cv.threshold(dist_transform,0.7*dist_transform.max(),255,0)
        # Finding unknown region
        sure_fg = np.uint8(sure_fg)
        unknown = cv.subtract(sure_bg,sure_fg)
        
        # Marker labelling
        ret, markers = cv.connectedComponents(sure_fg)
        # Add one to all labels so that sure background is not 0, but 1
        markers = markers+1
        # Now, mark the region of unknown with zero
        markers[unknown==255] = 0
        
        plt.imshow(markers,cmap = 'gray')
        plt.title('Grey markers Image'), plt.xticks([]), plt.yticks([])
        plt.show()
        
        col = cv.cvtColor(img,cv.COLOR_GRAY2BGR)
        
        
        markers = cv.watershed(col,markers)
        
        col[markers == -1] = [255,0,0]
        
        plt.imshow(col)
        plt.title('Watershed Image'), plt.xticks([]), plt.yticks([])
        plt.show()



In [ ]:
for idx, im_path in enumerate(tif_filelist):
    if idx != 0:
        break
    with Image.open(im_path) as im:
        # Takes a bit of time to display.
        #display(im)
        gray = img = cv.imread(im_path,0)

        # converting the image into gray-scale
        #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img2 = cv.medianBlur(img, 1)

        #applying adaptive threshold to use it as a mask
        edges = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 9, 9)

        color = cv.bilateralFilter(img, 9, 200, 200)

        #cartoonize
        cartoon = cv.bitwise_and(color, color, mask = edges)

        plt.imshow(color)
        plt.title('color Image'), plt.xticks([]), plt.yticks([])
        plt.show()

        